In [1]:
import requests

from bs4 import BeautifulSoup
from joblib import Parallel, delayed
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from datetime import datetime, timedelta, date

from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

from time import sleep

# Attempting to hijack session for direct graphql access

In [2]:
opts = Options()
opts.add_argument("--headless")
driver = Chrome(opts)


driver.get("https://www.aljazeera.com/search/katar%20football%20after:2020")
driver.get("https://www.aljazeera.com/graphql?wp-site=aje&operationName=SearchQuery&variables=%7B%22query%22%3A%22katar%20football%20after%3A2020%20before%3A2022%22%2C%22start%22%3A11%2C%22sort%22%3A%22relevance%22%7D&extensions=%7B%7D")
selenium_cookies = driver.get_cookies()

In [3]:

s = requests.Session()
for cookie in selenium_cookies:
    s.cookies.set(cookie['name'], cookie['value'])
res = s.get("https://www.aljazeera.com/search/katar%20football%20after:2020%20before:2022")
print(s.cookies.get_dict())
print(res.status_code)
res.raise_for_status()

{'AMP_9e2bdeb55f': 'JTdCJTIyZGV2aWNlSWQlMjIlM0ElMjIzYjA3MGY2Ni1kMmU5LTQ5ZGYtOGY2My1jM2Q3Y2Q3YjBhYWIlMjIlMkMlMjJzZXNzaW9uSWQlMjIlM0ExNzA1MjM0NzMyMzgyJTJDJTIyb3B0T3V0JTIyJTNBZmFsc2UlMkMlMjJsYXN0RXZlbnRUaW1lJTIyJTNBMTcwNTIzNDczMzA1NCUyQyUyMmxhc3RFdmVudElkJTIyJTNBMSU3RA==', 'OptanonConsent': 'isGpcEnabled=0&datestamp=Sun+Jan+14+2024+13%3A18%3A53+GMT%2B0100+(Central+European+Standard+Time)&version=202310.2.0&browserGpcFlag=0&isIABGlobal=false&hosts=&consentId=9349c44d-65f3-43af-aa3e-92cb98cda7c9&interactionCount=1&landingPath=https%3A%2F%2Fwww.aljazeera.com%2Fsearch%2Fkatar%2520football%2520after%3A2020&groups=C0001%3A1%2CC0007%3A1%2CC0002%3A0%2CC0003%3A0%2CC0004%3A0%2CC0005%3A0%2CV2STACK42%3A0', '_cb': 'D0vu4jDB__zSCc_wdN', '_cb_svref': 'external', '_chartbeat2': '.1705234732226.1705234732226.1.DSPloWCQpWJ4C8IJOVC5RM_4Dw3tDY.1', '_chartbeat4': 't=DOJNVkBwghh0Bd7jvEBPS528Hp1Qd&E=1&x=0&c=0.02&y=1242&w=600', '_ga': 'GA1.2.239982196.1705234732', '_ga_XN9JB9Q0M1': 'GS1.1.1705234732.1.0.17052347

In [4]:
json_url = "https://www.aljazeera.com/graphql?wp-site=aje&operationName=SearchQuery&variables=%7B%22query%22%3A%22katar%20football%20after%3A2020%20before%3A2022%22%2C%22start%22%3A1%2C%22sort%22%3A%22relevance%22%7D&extensions=%7B%7D"
res = s.get(json_url)
print(res.status_code)

403


One more idea: I have learned that al jazeera supports before: and after: statements in their search. It is extremely extremely unlikely that they posted more than the first page of articles in one day. We could send requests throughout one day.

In [5]:
import urllib.parse
urllib.parse.unquote("katar%20football%20after:2020-01-02%20before:2022")

'katar football after:2020-01-02 before:2022'

In [73]:
# Time range generators

def generate_daily_dates(start_date_str, end_date_str):
    """
    Generate a list of daily ISO formatted dates between the given start and end dates.

    Parameters:
    - start_date_str (str): Start date in the format 'YYYY-MM-DD'.
    - end_date_str (str): End date in the format 'YYYY-MM-DD'.

    Returns:
    - List of ISO formatted dates.
    """
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")

    # Generate a list of daily ISO formatted dates
    iso_dates = [(start_date + timedelta(days=x)).strftime("%Y-%m-%d") for x in range((end_date - start_date).days + 1)]

    return iso_dates



def get_week_ranges(start_date_str, end_date_str):
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d") - timedelta(days=datetime.strptime(start_date_str, "%Y-%m-%d").weekday())
    
    while start_date <= datetime.strptime(end_date_str, "%Y-%m-%d"):
        end_date = start_date + timedelta(days=6)
        yield start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d")
        start_date += timedelta(days=7)

def get_month_ranges(start_date_str, end_date_str):
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d").replace(day=1)

    while start_date <= datetime.strptime(end_date_str, "%Y-%m-%d"):
        end_date = (start_date + timedelta(days=32)).replace(day=1) - timedelta(days=1)
        yield start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d")
        start_date = (end_date + timedelta(days=1)).replace(day=1)

In [74]:

# Non-Scrolling link crawler


def get_articles_in_daterange(driver, search_terms, start_date, end_date, retry_count = 5, failed_daterange_log: list = None):
    """
    Al Jazeera demands distinct dates, but search range is inclusive. Thus, the minimum amount of days that can be searched is 2.

    Args:
        driver: Selenium Driver to use. Try to use a separate one for each parallel task.
        search_terms: String with space separated search terms. Both will occur in each result.
        start_date: Date in form YYYY_MM_DD. Articles from this day are included in the results.
        end_date: Date in form YYYY_MM_DD. Articles from this day are included in results.
    """

    if not failed_daterange_log:
        failed_daterange_log = []
    base_url = "https://www.aljazeera.com"
    full_search_query = f"{search_terms} after:{start_date}"
    if end_date:
        full_search_query += f" before:{end_date}"

    url = f"https://www.aljazeera.com/search/{full_search_query.replace(' ', '%20')}"

    driver.get(url)

    # Wait for the page to load (adjust the timeout as needed)
    failure_counter = 0
    success = False
    while (not success) and (failure_counter < 5):
        try:
            WebDriverWait(driver, 3).until(
                EC.any_of(
                    EC.presence_of_element_located((By.CLASS_NAME, "u-clickable-card__link")), # Results are here
                    EC.presence_of_element_located((By.CLASS_NAME, "search-results__no-results")) # No results found
                )
            )
            success = True
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            link_elements = soup.find_all("a", class_="u-clickable-card__link")

            links = set()
            for a_tag in link_elements:
                if base_url not in a_tag["href"]:
                    links.add(f'{base_url}/{a_tag["href"]}')
                else:
                    links.add(a_tag["href"])
            links = list(links)
            print(f"{start_date} - {end_date}, {len(links)} article links found.")
            if len(links)>=10:
                print(f"{start_date} - {end_date}, More than 10 links found. Consider refining this search, our current method can only fetch the first page.")
                failed_daterange_log.append((start_date, end_date, url,"OVERFLOW"))
            
        except TimeoutException: 
            # Timing out means that none of the above conditions was met. 
            # This could indicate a server rejection due to spam. 
            # We can not check as we are forced to use selenium.
            failure_counter += 1

            if "403" in str(driver.page_source): 
                print("Unauthorized detected. Going to sleep...")
            # Escalating wait time to try and trick spam rage
                sleep(2 ** (failure_counter + 1))

            print(f"{start_date} - {end_date}: Timed out for the {failure_counter} th time.")
    if failure_counter >= 5:
        print(f"{start_date} - {end_date} failed {failure_counter} times. giving up on this time range.")
        failed_daterange_log.append((start_date, end_date, "FAILED"))
        links = []
    return links

def get_articles_in_month(search_terms, start_date):
    opitons = Options()
    opitons.add_argument("--headless")
    driver = Chrome(opitons)

    failed_daterange_log = []
    month_start_date = datetime.strptime(start_date, "%Y-%m-%d")
    month_end_date = (month_start_date + timedelta(days=32)).replace(day=1) - timedelta(days=1)

    month_links = get_articles_in_daterange(driver, search_terms, month_start_date.strftime("%Y-%m-%d"), month_end_date.strftime("%Y-%m-%d"), failed_daterange_log=failed_daterange_log)

    if len(month_links) > 5:
        month_links = []
        print(f"more than 5 links found in MONTH {month_start_date.strftime('%Y-%m-%d')} : {month_end_date.strftime('%Y-%m-%d')}. Looking with finer granularity...")
        # If more than one link is found, recursively get articles for each week
        for week_start_date, week_end_date in get_week_ranges(month_start_date.strftime("%Y-%m-%d"), month_end_date.strftime("%Y-%m-%d")):
            month_links.extend(get_articles_in_week(search_terms, week_start_date, driver, failed_daterange_log)[0])
    del driver
    return month_links, failed_daterange_log

def get_articles_in_week(search_terms, start_date, driver, daterange_log):
    week_start_date = datetime.strptime(start_date, "%Y-%m-%d")
    week_end_date = week_start_date + timedelta(days=6)

    week_links = get_articles_in_daterange(driver, search_terms, week_start_date.strftime("%Y-%m-%d"), week_end_date.strftime("%Y-%m-%d"), failed_daterange_log=daterange_log)

    if len(week_links) > 5:
        week_links = []
        print(f"more than 5 links found in WEEK {week_start_date.strftime('%Y-%m-%d')} : {week_end_date.strftime('%Y-%m-%d')}. Looking with finer granularity...")
        # If more than one link is found, recursively get articles for each day
        for day_date in generate_daily_dates(week_start_date.strftime("%Y-%m-%d"), week_end_date.strftime("%Y-%m-%d")):
            week_links.extend(
                get_articles_in_daterange(
                    driver, 
                    search_terms, 
                    day_date, 
                    (datetime.strptime(day_date, '%Y-%m-%d') + timedelta(days=1))
                        .strftime('%Y-%m-%d'),
                    failed_daterange_log=daterange_log
                        )
            )
    return week_links

def collect_article_links(search_terms, start_date_str, end_date_str):
    month_ranges = get_month_ranges(start_date_str, end_date_str)
    month_links_logs = Parallel(-1, return_as="list")(delayed(lambda start_date: get_articles_in_month(search_terms, start_date))(start_date) for start_date, end_date in month_ranges)
    links = []
    log_agg = []
    for linklist, log_atom in month_links_logs:
        links.extend(linklist)
        log_agg.extend(log_atom)
    return list(set(links)), log_agg # Unique


In [11]:
links, daterange_log = collect_article_links("katar worldcup","2020-01-01", date.today().strftime("%Y-%m-%d"))

2020-01-01 - 2020-01-31, 1 article links found.
2020-08-01 - 2020-08-31, 1 article links found.
2020-04-01 - 2020-04-30, 0 article links found.
2020-05-01 - 2020-05-31, 0 article links found.
2020-06-01 - 2020-06-30, 2 article links found.
2020-02-01 - 2020-02-29, 0 article links found.
2020-07-01 - 2020-07-31, 0 article links found.
2020-03-01 - 2020-03-31, 0 article links found.
2020-11-01 - 2020-11-30: Timed out for the 1 th time.
2020-12-01 - 2020-12-31: Timed out for the 1 th time.
2021-03-01 - 2021-03-31: Timed out for the 1 th time.
2020-10-01 - 2020-10-31: Timed out for the 1 th time.
2021-02-01 - 2021-02-28: Timed out for the 1 th time.2021-01-01 - 2021-01-31: Timed out for the 1 th time.

2021-04-01 - 2021-04-30: Timed out for the 1 th time.
2020-09-01 - 2020-09-30: Timed out for the 1 th time.
2020-12-01 - 2020-12-31, 1 article links found.
2021-02-01 - 2021-02-28, 0 article links found.
2021-04-01 - 2021-04-30, 0 article links found.
2020-09-01 - 2020-09-30, 0 article links

IndexError: list index out of range

In [ ]:

def print_article_metainfo(links):

    for i in links:
        resp = requests.get(i)
        if resp.status_code == 200:
            soup = BeautifulSoup(resp.content)
            date_ = soup.find("div", "date-simple")
            if date_:
                date_ = date_.find("span", attrs={"aria-hidden": "true"}).text
            else:
                print(i, soup.findAll("td", string="date"))
            print(date_, soup.title.text)
        else:
            print(resp.status_code, i)


20 Jan 2020 Qatar to build new solar power plant | Qatar World Cup 2022 | Al Jazeera
5 Jun 2020 Qatar blockade: Five things to know about the Gulf crisis | GCC News | Al Jazeera
5 Jun 2020 Understanding the blockade against Qatar | GCC News | Al Jazeera
30 Aug 2020 Minimum wage, no NOC: Qatar announces changes to labour law | Business and Economy | Al Jazeera
7 Oct 2020 Turkey’s Erdogan arrives in Qatar for talks with Sheikh Tamim | Recep Tayyip Erdogan News | Al Jazeera
18 Dec 2020 Qatar 2022: Football World Cup stadiums at a glance | Qatar World Cup 2022 News | Al Jazeera
8 Jan 2021 Qatar-GCC agreement: A victory for measured diplomacy | GCC | Al Jazeera
31 Jul 2021 Weightlifter Fares Elbakh wins Qatar’s first Olympic gold | Olympics News | Al Jazeera
27 Aug 2021 The Afghan minister who became a bicycle courier in Germany | News | Al Jazeera
23 Aug 2021 Afghans in Qatar share hopes and fears after fleeing the Taliban | Migration News | Al Jazeera
11 Aug 2021 Qatar appoints first amba

In [ ]:
from datetime import date
links_2, daterange_log_2= collect_article_links("katar FIFA","2020-01-01", date.today().strftime("%Y-%m-%d"))

10796.49s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
10796.67s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
10796.85s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
10797.03s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
10797.21s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
10797.39s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
10797.57s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
10797.75s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


2020-03-01 - 2020-03-31, 0 article links found.
2020-07-01 - 2020-07-31, 0 article links found.
2020-06-01 - 2020-06-30, 0 article links found.
2020-04-01 - 2020-04-30, 0 article links found.
2020-05-01 - 2020-05-31, 0 article links found.
2020-02-01 - 2020-02-29, 0 article links found.
2020-08-01 - 2020-08-31, 1 article links found.
2020-01-01 - 2020-01-31, 1 article links found.
2021-03-01 - 2021-03-31, 0 article links found.
2020-09-01 - 2020-09-30, 0 article links found.
2020-10-01 - 2020-10-31, 0 article links found.
2020-11-01 - 2020-11-30, 0 article links found.
2021-02-01 - 2021-02-28, 0 article links found.
2021-01-01 - 2021-01-31, 1 article links found.
2020-12-01 - 2020-12-31, 1 article links found.
2021-04-01 - 2021-04-30, 0 article links found.
2021-06-01 - 2021-06-30, 0 article links found.
2021-05-01 - 2021-05-31, 0 article links found.
2021-08-01 - 2021-08-31, 2 article links found.
2021-07-01 - 2021-07-31, 1 article links found.
2021-09-01 - 2021-09-30, 0 article links

Loading animation is our friend. This is one way to identify wether content has been reloaded. Investiagtion revealed that it disappears after articles have been reloaded.

<div class="loading-animation"><div class="screen-reader-text" aria-live="assertive" role="status" tabindex="-1">loading</div><svg width="80px" height="80px" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 100 100" preserveAspectRatio="xMidYMid"><g transform="translate(20 50)"><circle cx="0" cy="0" r="7" fill="#cccccc" fill-opacity="1" transform="scale(0.99275 0.99275)"><animateTransform attributeName="transform" type="scale" begin="-0.375s" calcMode="spline" keySplines="0.3 0 0.7 1;0.3 0 0.7 1" values="0;1;0" keyTimes="0;0.5;1" dur="1s" repeatCount="indefinite"></animateTransform></circle></g><g transform="translate(40 50)"><circle cx="0" cy="0" r="7" fill="#cccccc" fill-opacity="0.8" transform="scale(0.773605 0.773605)"><animateTransform attributeName="transform" type="scale" begin="-0.25s" calcMode="spline" keySplines="0.3 0 0.7 1;0.3 0 0.7 1" values="0;1;0" keyTimes="0;0.5;1" dur="1s" repeatCount="indefinite"></animateTransform></circle></g><g transform="translate(60 50)"><circle cx="0" cy="0" r="7" fill="#cccccc" fill-opacity="0.6" transform="scale(0.42525 0.42525)"><animateTransform attributeName="transform" type="scale" begin="-0.125s" calcMode="spline" keySplines="0.3 0 0.7 1;0.3 0 0.7 1" values="0;1;0" keyTimes="0;0.5;1" dur="1s" repeatCount="indefinite"></animateTransform></circle></g><g transform="translate(80 50)"><circle cx="0" cy="0" r="7" fill="#cccccc" fill-opacity="0.4" transform="scale(0.113418 0.113418)"><animateTransform attributeName="transform" type="scale" begin="0s" calcMode="spline" keySplines="0.3 0 0.7 1;0.3 0 0.7 1" values="0;1;0" keyTimes="0;0.5;1" dur="1s" repeatCount="indefinite"></animateTransform></circle></g></svg></div>

In [98]:
def are_dom_updates_complete(driver):
    return driver.execute_script("return (document.readyState === 'complete');")

def get_morebutton(driver):
    try:
        more_button = driver.find_element(By.CLASS_NAME, ("show-more-button"))
    except NoSuchElementException:
        more_button = None
    return more_button

def wait_until_results_loaded(driver, timeout=3):

    # Waiting for show_more_button to disappear and loading to appear
    WebDriverWait(driver, timeout).until(
            EC.all_of(
                EC.none_of(
                    EC.presence_of_element_located( 
                        (
                            By.CLASS_NAME,
                            "show-more-button"
                        )
                    )
                ),
                EC.presence_of_element_located( 
                    (
                        By.CLASS_NAME,
                        "loading-animation"
                    )
                ),
                are_dom_updates_complete

            )
        )
    
    # waiting for loading to disappear again, as well as jquery deactivation
    WebDriverWait(driver, timeout).until(
        EC.all_of(
            EC.none_of(
                EC.presence_of_element_located( 
                    (
                        By.CLASS_NAME,
                        "loading-animation"
                    )
                )
            ),
            are_dom_updates_complete
        )
    )


def render_complete_results(driver, url, override_overflow=False):
    driver.get(url)
    WebDriverWait(driver, 3).until(
                EC.any_of(
                    EC.presence_of_element_located(
                        (By.CLASS_NAME, "u-clickable-card__link")
                        ), # Results are here
                    EC.presence_of_element_located(
                        (By.CLASS_NAME, "search-results__no-results")
                        ) # No results found
                )
            )
    intermediate_soup = BeautifulSoup(driver.page_source, 'html.parser')
    amount = intermediate_soup.find("span", class_="search-summary__query")

    if "Top 100" in amount.text:
        if override_overflow:
            f"OVERRIDING: More than 100 links found. Consider refining this search, our current method can only fetch the first page. Search Page can't offer more."
        else:
            raise OverflowError(f"More than 100 links found. Consider refining this search, our current method can only fetch the first page.")
    
    # Shape of input: "About {number} results"
    found_link_amount = int(amount.text.split(" ")[1])
    more_button = get_morebutton(driver)
    link_elements = intermediate_soup.find_all("a", class_="u-clickable-card__link")
    prev_scroll_link_amount = len(link_elements)
    overscroll_counter = 0

    while more_button and prev_scroll_link_amount < found_link_amount and overscroll_counter < 20: # Edge case: If exactly 10 links are found, searchbutton re-appears but does not load new elements.
        driver.execute_script("arguments[0].click();", more_button)
        # Wait until show more button is gone (to avoid acc. Click spamming)
        wait_until_results_loaded(driver, timeout=3)
        more_button = get_morebutton(driver)
        intermediate_soup = BeautifulSoup(driver.page_source, 'html.parser')
        link_elements = intermediate_soup.find_all("a", class_="u-clickable-card__link")
        
        if len(link_elements) <= prev_scroll_link_amount:
            print(f"Scrolling {url}, link elements did not increase on scroll.")
            overscroll_counter += 1

        prev_scroll_link_amount = len(link_elements)

def get_articles_in_daterange_scrolling(driver, search_terms, start_date, end_date, retry_count = 5, failed_daterange_log: list = None, override_overflow=False):
    """
    Al Jazeera demands distinct dates, but search range is inclusive. Thus, the minimum amount of days that can be searched is 2.

    Args:
        driver: Selenium Driver to use. Try to use a separate one for each parallel task.
        search_terms: String with space separated search terms. Both will occur in each result.
        start_date: Date in form YYYY_MM_DD. Articles from this day are included in the results.
        end_date: Date in form YYYY_MM_DD. Articles from this day are included in results.
    """
    if not failed_daterange_log:
        failed_daterange_log = []
    base_url = "https://www.aljazeera.com"
    full_search_query = f"{search_terms} after:{start_date} before:{end_date}"
    url = f"https://www.aljazeera.com/search/{full_search_query.replace(' ', '%20')}"

    links = [] # Define as Fallback if necessary

    # Wait for the page to load (adjust the timeout as needed)
    failure_counter = 0
    success = False
    while (not success) and (failure_counter < 5):
        try:
            render_complete_results(driver, url, override_overflow=override_overflow) 
            soup = BeautifulSoup(driver.page_source)
            link_elements = soup.find_all("a", class_="u-clickable-card__link")
            links = set()

            amount = int(soup.find("span", class_="search-summary__query").text.split(" ")[1])
            if len(link_elements) <= amount:
                failed_daterange_log.append((start_date, end_date, "INCOMPLETE", url))
            for a_tag in link_elements:
                if base_url not in a_tag["href"]:
                    links.add(f'{base_url}/{a_tag["href"]}')
                else:
                    links.add(a_tag["href"])
            links = list(links)
            print(f"{start_date} - {end_date}, {len(link_elements)} article links found, {len(links)} unique. Expected: {soup.find('span', class_='search-summary__query').text}")
            success = True
        except OverflowError:
            # Pass on with better info
            raise OverflowError(f"{start_date} : {end_date}: Overflow.")
        except TimeoutException: 
            # Timing out means that none of the above conditions was met. 
            # This could indicate a server rejection due to spam. 
            # We can not check as we are forced to use selenium.
            failure_counter += 1
            if "403" in str(driver.page_source): 
                print("Unauthorized detected. Going to sleep...")
            # Escalating wait time to try and trick spam rage
                sleep(2 ** (failure_counter + 1))
            print(f"{start_date} - {end_date}: Timed out for the {failure_counter} th time.")
        
            if failure_counter >= 5:
                failed_daterange_log.append((start_date, end_date, "FAILED", url))
                raise TimeoutException(f"{start_date} - {end_date} failed {failure_counter} times. giving up on this time range.")
    if override_overflow:
        failed_daterange_log.append((start_date, end_date, "OVERFLOW", url))

    
    return links


def get_articles_in_month(search_terms, start_date):
    opitons = Options()
    opitons.add_argument("--headless")
    driver = Chrome(opitons)

    failed_daterange_log = []
    month_start_date = datetime.strptime(start_date, "%Y-%m-%d")
    month_end_date = (month_start_date + timedelta(days=32)).replace(day=1) - timedelta(days=1)


    try:
        month_links = get_articles_in_daterange_scrolling(driver, search_terms, month_start_date.strftime("%Y-%m-%d"), month_end_date.strftime("%Y-%m-%d"), failed_daterange_log=failed_daterange_log)
    except OverflowError:
        month_links = []
        print(f"{month_start_date.strftime('%Y-%m-%d')} more than max links found in MONTH : {month_end_date.strftime('%Y-%m-%d')}. Looking with finer granularity...")
        # If more than max displayed links are found, sample down into weeks
        for week_start_date, week_end_date in get_week_ranges(month_start_date.strftime("%Y-%m-%d"), month_end_date.strftime("%Y-%m-%d")):
            month_links.extend(get_articles_in_week(search_terms, week_start_date, driver, failed_daterange_log))

    except TimeoutException as e:
        month_links = [] # Surrogate
        print(e.msg)
    del driver
    return month_links, failed_daterange_log

def get_articles_in_week(search_terms, start_date, driver, daterange_log):
    week_start_date = datetime.strptime(start_date, "%Y-%m-%d")
    week_end_date = week_start_date + timedelta(days=6)

    try:
        week_links = get_articles_in_daterange_scrolling(driver, search_terms, week_start_date.strftime("%Y-%m-%d"), week_end_date.strftime("%Y-%m-%d"), failed_daterange_log=daterange_log)
    except TimeoutException as e:
        week_links = []
        print(e.msg)
    except OverflowError:
        week_links = []
        print(f"more than max links found in WEEK {week_start_date.strftime('%Y-%m-%d')} : {week_end_date.strftime('%Y-%m-%d')}. Looking with finer granularity...")
        # If more than one link is found, recursively get articles for each day
        for day_date in generate_daily_dates(week_start_date.strftime("%Y-%m-%d"), week_end_date.strftime("%Y-%m-%d")):
            try:
                week_links.extend(
                    get_articles_in_daterange_scrolling(
                        driver, 
                        search_terms, 
                        day_date, 
                        (datetime.strptime(day_date, '%Y-%m-%d') + timedelta(days=1))
                            .strftime('%Y-%m-%d'),
                        failed_daterange_log=daterange_log,
                        override_overflow=True # override overflow, we have reached minimal resolution
                            )
                )
            except TimeoutException as e:
                print(e.msg)
    return week_links

def collect_article_links(search_terms, start_date_str, end_date_str):
    month_ranges = get_month_ranges(start_date_str, end_date_str)
    month_links_logs = Parallel(-1, return_as="list")(delayed(lambda start_date: get_articles_in_month(search_terms, start_date))(start_date) for start_date, end_date in month_ranges)
    links = []
    log_agg = []
    for linklist, log_atom in month_links_logs:
        links.extend(linklist)
        log_agg.extend(log_atom)
    return list(set(links)), log_agg # Unique


In [99]:
art_links, log = collect_article_links("qatar world cup", "2020-01-01", "2023-12-31")

10757.32s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
10757.50s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
10757.67s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
10757.85s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
10758.04s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
10758.21s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
10758.39s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
10758.57s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


2020-02-01 - 2020-02-29, 6 article links found, 6 unique. Expected: About 6 results
2020-01-01 - 2020-01-31, 7 article links found, 7 unique. Expected: About 7 results
2020-03-01 - 2020-03-31, 8 article links found, 8 unique. Expected: About 8 results
2020-05-01 - 2020-05-31, 5 article links found, 5 unique. Expected: About 5 results
2020-04-01 - 2020-04-30, 8 article links found, 8 unique. Expected: About 8 results
2020-06-01 - 2020-06-30, 14 article links found, 14 unique. Expected: About 14 results
2020-08-01 - 2020-08-31, 15 article links found, 15 unique. Expected: About 15 results
2020-09-01 - 2020-09-30, 9 article links found, 9 unique. Expected: About 9 results
2020-10-01 - 2020-10-31, 17 article links found, 17 unique. Expected: About 17 results
Scrolling https://www.aljazeera.com/search/qatar%20world%20cup%20after:2020-11-01%20before:2020-11-30, link elements did not increase on scroll.
Scrolling https://www.aljazeera.com/search/qatar%20world%20cup%20after:2020-11-01%20before

In [104]:
len(set(art_links))

1819